In [55]:
import lex
import eval
import math
import com
import time

from pathlib import Path

In [2]:
def reproduce_int_sequence(i: int, alphabet_size: int):
    result = []
    while i != 0:
        d, m = divmod(i, alphabet_size)
        result.append(m)
        i = d

    result.reverse()

    return result

def compact_int_sequence(sequence: list[int], alphabet_size: int):
    it = iter(sequence)
    result = next(it)
    for s in it:
        result = result * alphabet_size + s

    return result


def create_alphabet(sequence: list[str]):
    alphabet = list(set(sequence))
    if len(alphabet) == 1:
        assert '*' not in alphabet
        alphabet.append('*')

    if alphabet.index(sequence[0]) == 0:
        alphabet.append(alphabet.pop(0))

    return alphabet

def compact_sequence(sequence: list[str]) -> tuple[int, list[str]]:
    alphabet = create_alphabet(sequence)

    result = compact_int_sequence(
        (
            alphabet.index(s)
            for s in sequence
        ),
        len(alphabet)
    )

    return result, alphabet


def parse_compact_sequence(compact: int, alphabet: list[str]):
    return [
        alphabet[i]
        for i in reproduce_int_sequence(compact, len(alphabet))
    ]

def compact_to_expr(compact: int, alphabet: list[str]):
    alphabet_size = len(alphabet)

    compact_encoded = lex.encode_int(compact)
    alphabet_encoded = lex.encode_string(alphabet)
    alphabet_size_encoded = lex.encode_int(alphabet_size)

    return f'''B$ B$ L" B$ L# B$ v" B$ v# v# L# B$ v" B$ v# v# L" L# ? B= v# I! S B. B$ v" B/ v# I{alphabet_size_encoded} BT I" BD B% v# I{alphabet_size_encoded} S{alphabet_encoded} I{compact_encoded}'''


def compact_string(string: str):
    compact, alphabet = compact_sequence(string)

    return compact_to_expr(compact, alphabet)


def prepare_lambdaman_result(task_i: int, expr):
    prefix = f'solve lambdaman{task_i} '
    prefix_encoded = lex.encode_string(prefix)

    return f'B. S{prefix_encoded} {expr}'


In [45]:
destination_dir = Path(f'solutions/lambdaman')
reported_destination_dir = destination_dir / 'reported'

def find_best_soultuion(i: int):
    solutions = list(destination_dir.glob(f'{i}_*'))
    if len(solutions) == 0:
        print(f'Lambdaman #{i} without solutions')
        return None

    best = min(solutions, key=lambda f: int(f.name.split('_')[1]))
    return best

def find_best_soultuion_compact(i: int):
    solutions = list(destination_dir.joinpath('compact').glob(f'{i}_*'))
    if len(solutions) == 0:
        print(f'Lambdaman #{i} without solutions')
        return None

    best = min(solutions, key=lambda f: int(f.name.split('_')[1]))
    return best


def get_best_solution_full(i: int):
    best = find_best_soultuion(i)

    if best is None:
        return None

    path = best.read_text().rstrip()

    return 'S' + lex.encode_string(f'solve lambdaman{i} {path}')


def get_best_solution_compact(i: int):
    best = find_best_soultuion_compact(i)

    if best is None:
        return None

    result = best.read_text().rstrip()

    return result


In [41]:
#for i in range(1, 22):
#    sol_path = find_best_soultuion(i)
#    if sol_path is None:
#        print(f'no solution for {i}')
#        continue

#    sol = sol_path.read_text().rstrip()

#    compact = prepare_lambdaman_result(i, compact_string(sol))

#    destination = destination_dir / 'compact' / f'{i}_{len(compact)}'
#    if destination.exists():
#        print(f'skip {i}')
#    else:
#        destination.write_text(compact)

In [57]:
for i in range(1, 22):
    best_full = get_best_solution_full(i)
    best_compact = get_best_solution_compact(i)

    if best_full is None and best_compact is None:
        continue

    best = min(
        [
            sol
            for sol in [best_full, best_compact]
            if sol is not None
        ]
        , key=lambda sol: len(sol)
    )

    print(f'{i} score: {len(best)} {best}')

    response_destination = Path(f'./solutions/lambdaman/reported/{i}_{len(best)}')
    response_destination.parent.mkdir(exist_ok=True)
    if response_destination.exists():
        print(f'skip {response_destination}')
    else:
        time.sleep(5)
        response = com.com(best, True)
        response_destination.write_text(response[0])


1 score: 33 S3/,6%},!-"$!-!.V}O>FFF>OLLLLLOLL
skip solutions/lambdaman/reported/1_33
2 score: 44 S3/,6%},!-"$!-!.W}L>OLL>>LLOO>>FFF>FF>>LLLOL
3 score: 58 S3/,6%},!-"$!-!.X}>>LLFOFF>FFOOLOLFFOLFOOLL>L>L>L>OOOFOLFF
4 score: 238 B. S3/,6%},!-"$!-!.Y} B$ B$ L" B$ L# B$ v" B$ v# v# L# B$ v" B$ v# v# L" L# ? B= v# I! S B. B$ v" B/ v# I% BT I" BD B% v# I% SF>OL Ie{/=:4};,7wLK?>R@zf}ZBfYW&}rTNUFIDH!1A0gNGoLmWoox[{U1R]*vREy\%mt>%?|ivry%D5Z*'x/A1{K6'*P6~"bBv\!Z0C9GfBvF^
5 score: 159 S3/,6%},!-"$!-!.Z}L>FFFOFOLOL>OLLL>L>>OOO>L>L>F>F>OF>FFFFFOF>OOF>OFOLOFLOLOLOLLLLLLL>L>OOOLL>F>L>F>L>>>F>>FFLOLL>>FFFFFFFFFFFFFFFOOL>LLFOFOFOOOOOOO>LOLLLF>F>F>
6 score: 73 B. S3/,6%},!-"$!-!.[} B$ Lf B$ vf B$ vf B$ vf SLLLLLLLL Lx B. vx B. vx vx
7 score: 254 B. S3/,6%},!-"$!-!.\} B$ B$ L" B$ L# B$ v" B$ v# v# L# B$ v" B$ v# v# L" L# ? B= v# I! S B. B$ v" B/ v# I% BT I" BD B% v# I% S>OLF I'mo6uNHCqw.:VDa]J&y2-(~NRh@_&7XrAI-0#Lon{Ad{Ns`ksbUoEvJ}[J{BXke'gdwA>I]$YDF^C_k=e5\tb\7n4u*ZfgW9aK-5HvYN`BQ#`_Wu%U4"+`j]qh
8 score: